## Bag of Words - BOW
    * 단어 수 (word count) 기반 :  column에 단어별로 index 부여, row 문장1, 문장2 ...
    * 단점
        - 문맥의미(semantic context) 반영 부족 : 보완하기 위해 n-gram(제한적)
        - 희소행렬 (sparse Matrix): 대부분의 값이 0으로 채워지는 행렬, 수행시간과 예측성능을 떨어뜨림 -> 기법마련
          밀집행렬 (Dense Matrix)  

### 픽처 벡터화 : M x N : M개의 문서, N개의 단어(픽처 = feature)
    * count 기반 
        - 단점 : 자주 사용되는 던어에 높은 값 부여 
    * TF-IDF (Term Frenquency - Inverse Document Frequency ) 기반
        - 개별문서에서 자주 사용되는 단어 높은 가중치를 주되,(Term Frenquency)
          모든 문서에서 전반적으로 자주 나타나는 단어 패널티 부여 (inverse Document Frequency)
           ==> 범용적 단어에 패널티 부여
$TFIDF_i = TF_i*log\cfrac{N}{DF_i}$
        - TFi = 개별 문서에서의 단어 i빈도
        - DFi = 단어 i를 가지고 있는 문서 개수
        - N = 전체 문서 개수

### 사이키런의 Count 및 TF-IDF 벡터화 구현 : CountVectorizer, TfidfVectorizer
    * CountVectorizer  : 소문자 일괄변환, 토큰화, 스톱워드 필터링 등의 전처리 수행 -> 피처 벡터화
        - 소문자일괄변환( lowercase=True )
        - 토큰화 (analyzer = True )
        - 스톱워드 필터링만 제공  cf)Stemmer, Lemmatize는 지원되지 않음 (별도함수만들거나 미리 Text Normalization 수행 필요)
        - 피처벡터화 파라미터
            + max_df = 100 : 100이하만 피처로 추출,  = 0.95 :95%까지만 추출 5% 추출하지 않음
            + min_df = 2 : 2이하는 제외, =0.02 : 2%이하는 추출하지 않음
            + max_features =2000 : 2000개의 피처만 추출
            + stop_words = 'english' : 영어의 스톱워드 제외
            + n_gram_range= (1,1),  = (1,2) : 최소 1개씩 순서대로 최대2개씩 피처로 추출
            + analyzer = 'word' : 피처추출을 수행한 단위, 'character' 특정범위를 피처로 만드는 경우
            + token_pattern = '\b\w\w+\b' : 공백 또는 개행문자 등으로 구분된 단어분리자(\b)사이의 
                                            2문자(문자 또는 숫자, 즉 영숫자)이상의 단어(word)를 토큰으로 분리, 변경은 거의없음
            + tokenizer = : 토큰화를 별도의 custom함수로 이용시 적용, 
                            일반적으로 CountTokenizer 클래스에서 어근변환시 이를 수행하는 별도의 함수를 적용
    * TF-IDF 벡터화는 TfidfVectorizer클래스 이용, 파라미터 동일
       
    

### 희소행렬
    * 너무많은 메모리 공간 차지 -> 적은 메모리 공간 차지를 위한 변환 필요
    * COO(Coordinate : 좌표) 방식
        - 0이 아닌 데이터만 별도의 데이터배열에 저장하고, 그 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식
        - 희소 행렬 변환을 위해서 사이파이(Scipy)
        - 

In [7]:
import numpy as np
dense = np.array( [ [ 3,0,1], [0, 2, 0]])

from scipy import sparse

# 0이 아닌 데이터 추출
data = np.array([3,1,2])

# (0,0), (0,2), (1,1)   행위치와 열 위치를 각각 배열로 생성  
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse패키지 coo_matrix를 이용해 coo형식으로 희소행령 생성
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))
sparse_coo 

<2x3 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in COOrdinate format>

In [5]:
sparse_coo.toarray() # 원래 데이터 행렬로 추출

array([[3, 0, 1],
       [0, 2, 0]])

## CSR 형식 (Compressed Sparse Row)
    * coo형식이 행과 열의 위치를 나타내기 위해 반복적인 위치 데이터 사용 => 해결
    * 행위치배열의 index 행렬, 개수 추가

In [25]:
import numpy as np
dense2 = np.array( [ [0,0,1,0,0,5],
                     [1,4,0,3,2,5],
                     [0,6,0,3,0,0],
                     [2,0,0,0,0,0],
                     [0,0,0,7,0,8],
                     [1,0,0,0,0,0]  ])

from scipy import sparse

# 0이 아닌 데이터 추출
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])

# (0,0), (0,2), (1,1)   행위치와 열 위치를 각각 배열로 생성  
row_pos = np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col_pos = np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

# sparse패키지 coo_matrix를 이용해 coo형식으로 희소행령 생성
sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))
print(f'sparse_coo:\n{sparse_coo}') # (0,2)  1   ==> (row,col) 0이 아닌 데이터 

# 행 위치 배열의 고유한 값의 시작위치 인덱스를 배열로 생성 # 마지막에 개수 추가
row_pos_ind = np.array( [0, 2, 7, 9, 10, 12, 12 ])

# CSR형식으로 변환
sparse_csr = sparse.csr_matrix( ( data2, col_pos, row_pos_ind))
print()
print(f'sparse_csr:\n{sparse_csr}') # (0,2)  1 ==> (row,col) 0이 아닌 데이터 


print(f'sparse_coo.toarray() COO변환된 데이터 다시 Dense 출력 : \n{sparse_coo.toarray()}\n')
print(f'sparse_csr.toarray() CSR변환된 데이터 다시 Dense 출력 : \n{sparse_csr.toarray()}')

sparse_coo:
  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1

sparse_csr:
  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
sparse_coo.toarray() COO변환된 데이터 다시 Dense 출력 : 
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]

sparse_csr.toarray() CSR변환된 데이터 다시 Dense 출력 : 
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [0 0 0 0 0 0]]


In [21]:
import numpy as np
dense3 = np.array( [ [0,0,1,0,0,5],
                     [1,4,0,3,2,5],
                     [0,6,0,3,0,0],
                     [2,0,0,0,0,0],
                     [0,0,0,7,0,8],
                     [1,0,0,0,0,0]  ])

from scipy import sparse
coo = sparse.coo_matrix(dense3)
print(coo) # (0,2)  1 ==> (row,col) 0이 아닌 데이터
csr = sparse.csr_matrix(dense3)
print()
print(csr)

  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1

  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1
